# Working with Images stored on the OMERO server using Cell Profiler

This notebook demonstrates how to retrieve Images stored in OMERO and process them using [CellProfiler](http://cellprofiler.org/). The output is saved back to OMERO as CSV attachments.
For this example, we use [FruitFlyCells](http://cellprofiler.org/examples/#FruitFlyCells).

### Workflow Summary
![Overview](images/fruit_fly_workflow.png)

### Import Packages

In [ ]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler.preferences as cpprefs
import cellprofiler.module as cpm
import cellprofiler.pipeline as cpp
cpprefs.set_headless()

# Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
from cellprofiler.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
from omero.gateway import BlitzGateway

# Import Numpy
import numpy as np

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

from getpass import getpass

### Set Cell Output Directory

In [ ]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials

In [ ]:
conn = BlitzGateway(host='ws://idr.openmicroscopy.org/omero-ws', username='public', passwd='public', secure=True)
conn.connect()

### Fetch OMERO Dataset that contains the Images to be analysed

In [ ]:
# To be modified
# Please fill in the datasetId for the dataset named 'CellProfiler images'
datasetid = 602
dataset = conn.getObject("Dataset", datasetid)
print 'Dataset Name: ',dataset.getName()

### Load pipeline and inspect modules

In [ ]:
from os.path import expanduser
home = expanduser("~")
pipeline = cpp.Pipeline()
pipeline.load(home+"/pipelines/ExampleFly.cppipe")

# Remove first 4 modules: Images, Metadata, NamesAndTypes, Groups...
# (replaced by InjectImage module below)
for i in range(4):
    print 'Remove module: ', pipeline.modules()[0].module_name
    pipeline.remove_module(1)
print 'Pipeline modules:'
for module in pipeline.modules():
    print module.module_num, module.module_name

### Run Cell Profiler Pipeline on the OMERO Images

In [ ]:
warnings.filterwarnings('ignore')

Nuclei = pandas.DataFrame()
Cells = pandas.DataFrame()
Cytoplasm = pandas.DataFrame()
Image = pandas.DataFrame()

for image in dataset.listChildren():

    pixels = image.getPrimaryPixels()
    size_c = image.getSizeC()
    # For each Image in OMERO, we copy pipeline and inject image modules
    pipeline_copy = pipeline.copy()
    for c in range(0, size_c):
        
        plane = pixels.getPlane(0, c, 0)
        image_name = image.getName()

        # Name of the channel expected in the pipeline
        if c == 0:
            Image_Name = 'OrigBlue'
        if c == 1:
            Image_Name = 'OrigGreen'
        if c == 2:
            Image_Name = 'OrigRed'

        inject_image_module = InjectImage(Image_Name, plane)
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)
    
    m = pipeline_copy.run()
    
    #Results obtained as CSV from Cell Profiler
    nuc_csv = pandas.read_csv(new_output_directory + '/Nuclei.csv')
    nuc_csv.ImageNumber = image.getName()
    Nuclei = Nuclei.append(nuc_csv)
    
    cel_csv = pandas.read_csv(new_output_directory + '/Cells.csv')
    cel_csv.ImageNumber = image.getName()
    Cells = Cells.append(cel_csv)
    
    cyto_csv = pandas.read_csv(new_output_directory + '/Cytoplasm.csv')
    cyto_csv.ImageNumber = image.getName()
    Cytoplasm = Cytoplasm.append(cyto_csv)
    
    image_csv = pandas.read_csv(new_output_directory + '/Image.csv')
    image_csv.ImageNumber = image.getName()
    Image = Image.append(image_csv)

## Results obtained as CSV from Cell Profiler

### Measurements (Nuclei)

In [ ]:
Nuclei.describe()

### Measurements (Cells)

In [ ]:
Cells.head()

### Measurements (Cytoplasm)

In [ ]:
Cytoplasm.head()

### Images (Summary)

In [ ]:
Image.head()

### Push Results back to OMERO

In [ ]:
import glob

for file_to_upload in glob.glob(new_output_directory + '/*.csv'):
    
    if file_to_upload.endswith('Nuclei.csv'):
        Nuclei.to_csv(file_to_upload)
    elif file_to_upload.endswith('Cells.csv'):
        Cells.to_csv(file_to_upload)
    elif file_to_upload.endswith('Cytoplasm.csv'):
        Cytoplasm.to_csv(file_to_upload)
    elif file_to_upload.endswith('Image.csv'):
        Image.to_csv(file_to_upload)
    
    # create the original file and file annotation (uploads the file etc.)
    namespace = "training.meeting.demo"
    print "\nCreating an OriginalFile and FileAnnotation"
    file_ann = conn.createFileAnnfromLocalFile(file_to_upload, mimetype="text/plain", ns=namespace, desc=None)
    print "Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(), \
        ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize()
    dataset.linkAnnotation(file_ann)

<br>
#### Close the connection to the OMERO server

In [ ]:
conn.close()